In [0]:
import requests
import pandas as pd
import time
from pyspark.sql.functions import current_timestamp

# --- 1. CONFIGURAÇÃO ---
API_KEY = "f4a68bb2393847b82a7b667d4f4ae537"  # <--- COLOQUE SUA CHAVE
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

# Lista completa das 27 capitais brasileiras
capitais = [
    "Rio Branco", "Maceió", "Macapá", "Manaus", "Salvador", "Fortaleza", "Brasília",
    "Vitória", "Goiânia", "São Luís", "Cuiabá", "Campo Grande", "Belo Horizonte",
    "Belém", "João Pessoa", "Curitiba", "Recife", "Teresina", "Rio de Janeiro",
    "Natal", "Porto Alegre", "Porto Velho", "Boa Vista", "Florianópolis",
    "São Paulo", "Aracaju", "Palmas"
]

lista_dados_coletados = [] # Lista vazia para guardar os resultados

print(f"Iniciando coleta de {len(capitais)} capitais...")

# --- 2. O LOOP (EXTRAÇÃO) ---
for cidade in capitais:
    
    # Montamos os parâmetros, forçando ",BR" para garantir que é no Brasil
    params = {
        "q": f"{cidade},BR", 
        "appid": API_KEY,
        "units": "metric",
        "lang": "pt_br"
    }
    
    try:
        response = requests.get(BASE_URL, params=params)
        
        if response.status_code == 200:
            data = response.json()
            
            # Extraindo só o que importa (Flattening)
            registro = {
                "cidade": data["name"],
                "estado_sigla": "BR", # A API as vezes não manda o estado preciso, mas sabemos que é BR
                "temperatura": data["main"]["temp"],
                "sensacao": data["main"]["feels_like"],
                "umidade": data["main"]["humidity"],
                "clima": data["weather"][0]["description"],
                "latitude": data["coord"]["lat"],
                "longitude": data["coord"]["lon"]
            }
            
            lista_dados_coletados.append(registro)
            print(f"✔ {cidade}: OK")
            
        else:
            print(f"❌ Erro em {cidade}: {response.status_code}")
            
    except Exception as e:
        print(f"⚠ Erro crítico em {cidade}: {str(e)}")

# --- 3. TRANSFORMAÇÃO E CARGA ---

# Só prossegue se tiver coletado algo
if lista_dados_coletados:
    
    # Cria o DataFrame Pandas com TODOS os dados de uma vez
    df_pandas = pd.DataFrame(lista_dados_coletados)
    
    # Converte para Spark
    df_spark = spark.createDataFrame(df_pandas)
    
    # Adiciona data de processamento
    df_final = df_spark.withColumn("data_ingestao", current_timestamp())
    
    # Exibe resultado
    display(df_final)
    
    # Salva no Delta Lake (agora especificando o lugar certo)
    # Formato: nome_do_schema.nome_da_tabela
    nome_completo_tabela = "aprender_data.tabela_clima_capitais"
    
    (df_final.write
        .format("delta")
        .mode("append")
        .option("mergeSchema", "true") # Garante que se a tabela mudar, ele aceita
        .saveAsTable(nome_completo_tabela))
    
    print(f"\nSUCESSO: Todas as capitais processadas e salvas em: {nome_completo_tabela}!")

In [0]:
%sql
-- Cria uma "pasta" chamada aprender_data se ela não existir
CREATE SCHEMA IF NOT EXISTS aprender_data;